# 라이브러리

In [339]:
import pymysql
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_squared_error

# 데이터 전처리

### SQL에서 불러온 데이터

##### 데이터 병합

In [332]:
# 일자 + 시간 : time

import pandas as pd
import pymysql
conn = pymysql.connect(host='127.0.0.1', user='root', password='kyjin0413!', db='lpointsample', charset='utf8')
cur = conn.cursor()
cur.execute("select de_dt as 날짜, de_hr as 시간대, avg(buy_am) as 금액 ,count(*) as 이용횟수 from 엘페이이용_view group by de_dt,de_hr order by de_dt,de_hr;")
result = cur.fetchall()
conn.commit()
conn.close()

time = pd.DataFrame(result)
time.columns=['일시','시간대', '소비 금액','이용 횟수']
df_date = time['일시'].values.tolist()
for i in range(len(time)):
    a = str(time['일시'][i])[:10]
    b = str(time['시간대'][i])
    if len(b) == 1:
        b= "0"+b
    time['일시'][i]=a+" "+b
    time['일시'][i] = pd.to_datetime(time['일시'][i], format='%Y-%m-%d %H')

time = time.drop(columns=['시간대'])

time

C:\Users\ASIA-08\AppData\Local\Temp\ipykernel_13080\2034004478.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time['일시'][i]=a+" "+b
C:\Users\ASIA-08\AppData\Local\Temp\ipykernel_13080\2034004478.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time['일시'][i] = pd.to_datetime(time['일시'][i], format='%Y-%m-%d %H')


,일시,소비 금액,이용 횟수
0,2021-01-01 00:00:00,37269.1667,72
1,2021-01-01 01:00:00,45396.3636,11
2,2021-01-01 02:00:00,6417.6000,10
3,2021-01-01 03:00:00,80530.2727,11
4,2021-01-01 05:00:00,37630.0000,1
...,...,...,...
8628,2021-12-31 19:00:00,81530.0000,63
8629,2021-12-31 20:00:00,43399.2917,72
8630,2021-12-31 21:00:00,36667.0000,33
8631,2021-12-31 22:00:00,29141.9792,48


### 결측치 처리 (없는 시간대에 0 추가)

In [345]:
# 결측치 제거 : time_final -> 4:00 추가

list_date = []
for v in df_date:
    if v in list_date:
        pass
    else:
        list_date.append(v)

list_time = []
col = []

for i in range(24):
    list_time.append(i)
for i in range(len(list_date)):
    a = str(list_date[i])[:10]
    for j in range(len(list_time)):
        b = str(list_time[j])
        if len(b) == 1:
            b= "0"+b
        col.append(a+" "+b)

category=['소비 금액','이용 횟수']
time_final = pd.DataFrame(index = col, columns=category)
time_final.index = pd.to_datetime(time_final.index, format='%Y-%m-%d %H')

for i in range(len(time)):
    time_final.loc[time['일시'][i]]['소비 금액'] = time['소비 금액'][i]
    time_final.loc[time['일시'][i]]['이용 횟수'] = time['이용 횟수'][i]

time_final = time_final.fillna(0)
time_final.reset_index(inplace=True)
time_final = time_final.rename(columns = {'index':'일시'})

time_final.head

,일시,소비 금액,이용 횟수
0,2021-01-01 00:00:00,37269.1667,72
1,2021-01-01 01:00:00,45396.3636,11
2,2021-01-01 02:00:00,6417.6000,10
3,2021-01-01 03:00:00,80530.2727,11
4,2021-01-01 04:00:00,0,0
5,2021-01-01 05:00:00,37630.0000,1


### CSV 파일 데이터

##### 기온과 습도 평균 구하기

In [346]:
# 온도습도 평균 구하는 전처리 코드 : temphum_final

temphum = pd.read_csv('온도습도.csv', encoding = 'cp949')
temphum = temphum.fillna(0)

df1 = temphum['일시']
dflist = df1.values.tolist()
date = []
for v in dflist:
    if v in date:
        pass
    else:
        date.append(v)

temp = temphum['기온(°C)'][temphum['일시'] == date[0]].tolist()

hum = temphum['습도(%)'][temphum['일시'] == date[0]].tolist()

temphum_final = pd.DataFrame(index=date, columns= ['평균 기온(°C)','평균 습도(%)'])

for v in date:
    temperature = temphum['기온(°C)'][temphum['일시'] == v].tolist()
    
    humidity = b = temphum['습도(%)'][temphum['일시'] == v].tolist()
    
    temphum_final.loc[v]['평균 기온(°C)']=sum(temperature)/len(temperature)
    
    temphum_final.loc[v]['평균 습도(%)']=sum(humidity)/len(humidity)

temphum_final.reset_index(inplace=True)
temphum_final = temphum_final.rename(columns = {'index':'일시'})
temphum_final = temphum_final.drop(['일시'], axis = 1)

temphum_final

,평균 기온(°C),평균 습도(%)
0,-4.855789,69.578947
1,-5.383158,70.536842
2,-5.887368,71.863158
3,-6.305263,72.842105
4,-6.789474,73.789474
...,...,...
8755,-3.010526,41.368421
8756,-3.637895,43.326316
8757,-4.266316,45.115789
8758,-4.897895,47.852632


##### 제공 데이터와 외부 데이터 병합

In [338]:
# 일자 기준으로 엘포인트 데이터와 기상청 데이터 병합 후 '일시' 인덱스로 지정
absorption = pd.concat([time_final, temphum_final], axis = 1)
absorption = absorption.set_index(['일시'])
absorption = absorption.astype('float64')
absorption.dtypes

absorption

,소비 금액,이용 횟수,평균 기온(°C),평균 습도(%)
일시,,,,
2021-01-01 00:00:00,37269.1667,72,-4.855789,69.578947
2021-01-01 01:00:00,45396.3636,11,-5.383158,70.536842
2021-01-01 02:00:00,6417.6000,10,-5.887368,71.863158
2021-01-01 03:00:00,80530.2727,11,-6.305263,72.842105
2021-01-01 04:00:00,0,0,-6.789474,73.789474
...,...,...,...,...
2021-12-31 19:00:00,81530.0000,63,-3.010526,41.368421
2021-12-31 20:00:00,43399.2917,72,-3.637895,43.326316
2021-12-31 21:00:00,36667.0000,33,-4.266316,45.115789


In [327]:
# 데이터 타입 변환
absorption = absorption.astype('float64')
absorption.dtypes

소비 금액    float64
dtype: object

# 모델 설계 - 그리드서치

In [340]:
# 그리드 서치(모델에게 가장 적합한 하이퍼 파라미터를 찾기)
import itertools

start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

# 기준 날짜에 대한 연산이 필요할 때 dateutil.relativedelta모듈의 relativedelta함수 이용
df_list = [start + relativedelta(hours = x) for x in range(0, len(absorption)) ]

p = d = q = range(0, 2)
pdq = list( itertools.product(p, d, q))
seasonal_pdq = [ (x[0], x[1], x[2], 24)  for x in pdq ]

select_candi = 10000000
param_candi = ( 0, 0, 0 )
param_seasonal_candi = ( 0, 0, 0)

count=0
end_count = len(pdq)

for param in pdq:   
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(absorption['소비 금액'],
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False
                                           )
            results = mod.fit()
            count += 1
            if count <= 5:
                print('SARIMA{}x{}24 - AIC:{}'.format(param, param_seasonal, results.aic))
            
            if results.aic < select_candi:
                select_candi = results.aic
                param_candi = param
                param_seasonal_candi = param_seasonal
        except:
            continue

print('----최적의 Oreder 값과 Seasonal_Order 값----')
print('Order : {}, Seasonal_Order : {}'.format(param_candi, param_seasonal_candi))

# 2021년 예측

In [328]:
# 2021년 예측을 위한 데이터
train21 = absorption[:8016]['소비 금액']

exog1 = absorption.drop('소비 금액', axis=1)[:8016]
exog1 = sm.add_constant(exog1)

exog2 = absorption.drop('소비 금액', axis=1)[8016:]
exog2 = sm.add_constant(exog2)

In [329]:
# 2021년 예측 값과 실제 값 비교 (12월 1일 00시 ~ 12월 31일 23시)

def predict(exog1, exog2):

    mod = sm.tsa.statespace.SARIMAX(train21, exog = exog1, 
                            order = (0, 0, 0), seasonal_order = (1, 1, 1, 24))
    fit_res = mod.fit(disp = False)

    forecast = fit_res.forecast(steps = 744, exog = exog2)

    # 2021년 실제값
    actual = absorption['소비 금액'][8016:]

    rmse = np.sqrt(mean_squared_error(actual, forecast))
    
    return rmse

print('----실제 값과 예측 값 비교 RMSE----')
print(predict(exog1, exog2))

----실제 값과 예측 값 비교 RMSE----


c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


56959.24587003265


In [330]:
# 구축한 모델로 2021년 소비 금액 예측 값 (12월 1일 00시 ~ 12월 31일 23시)
mod = sm.tsa.statespace.SARIMAX(train21, exog = exog1, 
                        order = (0, 0, 0), seasonal_order = (1, 1, 1, 24))

fit_res = mod.fit(disp = False)
forecast = fit_res.forecast(steps = 744, exog = exog2)

forecast.head(10)

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


2021-12-01 00:00:00    61524.470926
2021-12-01 01:00:00    52894.824235
2021-12-01 02:00:00    47841.507331
2021-12-01 03:00:00    56949.311755
2021-12-01 04:00:00    38846.908499
2021-12-01 05:00:00    49758.511071
2021-12-01 06:00:00    48429.261726
2021-12-01 07:00:00    34684.337425
2021-12-01 08:00:00    42019.921712
2021-12-01 09:00:00    51072.712474
Freq: H, Name: predicted_mean, dtype: float64

# 2022년 예측

In [342]:
# 2022년 예측을 위한 데이터

data2022 = pd.read_csv('최종.csv', encoding='cp949')

data2022 = data2022.set_index('일자')
data2022 = data2022.fillna(method='ffill')
data2022.tail()

,소비 금액,이용 횟수
일자,,
2022-12-31 19:00,52463.0,67
2022-12-31 20:00,43072.0,57
2022-12-31 21:00,41635.0,48
2022-12-31 22:00,43678.0,41
2022-12-31 23:00,58802.0,31


In [343]:
# 2022년 예측

train22 = data2022[:'2021-12-31 23:00']['소비 금액']

exog3 = data2022.drop('소비 금액', axis=1)[:'2021-12-31 23:00']

exog4 = data2022.drop('소비 금액', axis=1)['2022-01-01 0:00':]

In [344]:
# 구축한 모델로 2021년 소비 금액 예측 값 (12월 1일 00시 ~ 12월 31일 23시)
mod = sm.tsa.statespace.SARIMAX(train22, exog = exog3, 
                        order = (0, 0, 0), seasonal_order = (1, 1, 1, 24))

fit_res = mod.fit(disp = False)
forecast = fit_res.forecast(steps = 8760, exog = exog4)

forecast.head(10)

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


2022-01-01 00:00:00    58874.437536
2022-01-01 01:00:00    51710.826286
2022-01-01 02:00:00    48468.896150
2022-01-01 03:00:00    50825.594629
2022-01-01 04:00:00    43764.945679
2022-01-01 05:00:00    48829.674445
2022-01-01 06:00:00    51229.598603
2022-01-01 07:00:00    33803.558781
2022-01-01 08:00:00    43340.941776
2022-01-01 09:00:00    49870.150587
Freq: H, Name: predicted_mean, dtype: float64